# 地理空間データの可視化

## ポイントの可視化

In [ ]:
import pandas as pd

itami_wifi = pd.read_csv("data/282073publicwirelesslan202204.csv")
itami_wifi.iloc[:3, 3:10]

In [ ]:
itami_center = itami_wifi.loc[:, ["緯度", "経度"]].mean()
itami_center

In [ ]:
import folium

itami_wifi_map = folium.Map(
    location=itami_center,
    zoom_start=15.2,
    width=800,
    height=400,
)
itami_wifi_map

In [ ]:
marker = folium.Marker(location=itami_wifi.loc[0, ["緯度", "経度"]])
marker.add_to(itami_wifi_map)
itami_wifi_map

In [ ]:
itami_wifi.loc[:, ["緯度", "経度"]].apply(
    lambda x: folium.Marker(location=x).add_to(itami_wifi_map),
    axis=1,
)
itami_wifi_map

In [ ]:
import geopandas as gpd

land_price_gdf = (
    gpd.read_file(
        "https://nlftp.mlit.go.jp/ksj/gml/data/L01/L01-22/L01-22_25_GML.zip"
    )
    .groupby("L01_005")
    .get_group("2022")
)
land_price_gdf.loc[:3, ["L01_006", "L01_023", "geometry"]]

In [ ]:
land_price = land_price_gdf.apply(
    lambda s: pd.Series(
        (
            s["geometry"].y,
            s["geometry"].x,
            s["L01_006"],
        )
    ),
    axis=1,
).values
land_price[:3]

In [ ]:
from folium.plugins import HeatMap

shiga_center = (
    land_price_gdf.loc[:, "geometry"].y.mean(),
    land_price_gdf.loc[:, "geometry"].x.mean(),
)
land_price_map = folium.Map(
    location=shiga_center,
    zoom_start=10,
)
HeatMap(
    data=land_price,
    radius=12,
).add_to(land_price_map)
land_price_map

## ラインの可視化

In [ ]:
minx, miny = land_price_gdf.loc[:, "geometry"].bounds.min()[
    ["minx", "miny"]
]
maxx, maxy = land_price_gdf.loc[:, "geometry"].bounds.max()[
    ["maxx", "maxy"]
]
rail_gdf = gpd.read_file(
    "https://nlftp.mlit.go.jp/ksj/gml/data/N02/N02-21/N02-21_GML.zip"
).cx[minx:maxx, miny:maxy]
rail_gdf.head()

In [ ]:
# 京津線のデータを抽出
keishinsen = rail_gdf.groupby("N02_003")["geometry"].get_group("京津線")
# 中心となる座標
keishinsen_center = (
    keishinsen.map(lambda p: p.centroid.y).mean(),
    keishinsen.map(lambda p: p.centroid.x).mean(),
)
keishinsen_center

In [ ]:
import numpy as np


keishinsen_map = folium.Map(
    location=keishinsen_center,
    zoom_start=14,
)
keishinsen.apply(
    lambda s: folium.PolyLine(
        # 緯度, 経度の順番に変換、fliplr関数で順序を反転
        locations=np.fliplr(s.coords),
        weight=15  # 線の太さ
    ).add_to(keishinsen_map)
)
keishinsen_map

In [ ]:
rail_map = folium.Map(
    location=shiga_center,
    zoom_start=10,
)
folium.features.GeoJson(rail_gdf).add_to(rail_map)
rail_map

## ポリゴンの可視化

In [ ]:
did_gdf = gpd.read_file("./data/A16-15_25_DID.geojson")
did_gdf.head()

In [ ]:
did_map = folium.Map(
    location=shiga_center,
    zoom_start=10,
)
folium.GeoJson(did_gdf).add_to(did_map)
did_map

In [ ]:
did_choroplethmap = folium.Map(
    location=shiga_center,
    zoom_start=10,
)
folium.features.Choropleth(
    geo_data=did_gdf,
    data=did_gdf,
    key_on="feature.properties.DIDid",
    columns=["DIDid", "人口"],
    legend_name="人口",
).add_to(did_choroplethmap)
did_choroplethmap